# CounterFactual Experiment

## Import Libraries

In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import importlib
import itertools
import warnings
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.utils import Bunch
from typing import cast
from CounterFactualModel import CounterFactualModel
from ConstraintParser import ConstraintParser
import CounterFactualVisualizer as CounterFactualVisualizer
importlib.reload(CounterFactualVisualizer)
from CounterFactualVisualizer import (plot_pca_with_counterfactual, plot_sample_and_counterfactual_heatmap, 
                                     plot_pca_loadings, plot_constraints, 
                                     plot_sample_and_counterfactual_comparison, plot_pairwise_with_counterfactual_df,
                                     plot_pca_with_counterfactuals, plot_explainer_summary)
from CounterFactualExplainer import CounterFactualExplainer

warnings.filterwarnings("ignore")

## Setup + Constants

In [ ]:
CLASS_COLORS_LIST = ['purple', 'green', 'orange']
IRIS: Bunch = cast(Bunch, load_iris())
IRIS_FEATURES = IRIS.data
IRIS_LABELS = IRIS.target

TRAIN_FEATURES, TEST_FEATURES, TRAIN_LABELS, TEST_LABELS = train_test_split(IRIS_FEATURES, IRIS_LABELS, test_size=0.3, random_state=42)

MODEL = RandomForestClassifier(n_estimators=3, random_state=42)
MODEL.fit(TRAIN_FEATURES, TRAIN_LABELS)


ORIGINAL_SAMPLE = {'petal width (cm)': 6.1, 'petal length (cm)': 2.8, 'sepal length (cm)': 4.7, 'sepal width (cm)': 1.2}
SAMPLE_DATAFRAME = pd.DataFrame([ORIGINAL_SAMPLE])  # Wrapping sample in a list to create a DataFrame

CONSTRAINT_PARSER = ConstraintParser("constraints/custom_l100_pv0.001_t2_dpg_metrics.txt")
CONSTRAINTS = CONSTRAINT_PARSER.read_constraints_from_file()

COUNTERFACTUAL_DPG = CounterFactualModel(MODEL, CONSTRAINTS, verbose=True)  # Ensure you are using the correct constraints (nested_dict)
TARGET_CLASS = 0
ORIGINAL_SAMPLE_PREDICTED_CLASS = MODEL.predict(SAMPLE_DATAFRAME)

RULES = ['no_change', 'non_increasing', 'non_decreasing']
FEATURES_NAMES = list(ORIGINAL_SAMPLE.keys())
RULES_COMBINATIONS = list(itertools.product(RULES, repeat=len(FEATURES_NAMES)))

NUMBER_OF_COMBINATIONS_TO_TEST = len(RULES_COMBINATIONS)
NUMBER_OF_REPLICATIONS_PER_COMBINATION = 10
INITIAL_POPULATION_SIZE = 20
MAX_GENERATIONS = 60


## Constraints Extracted from DPG

In [ ]:

plot_constraints(CONSTRAINTS, overlapping=False, class_colors_list=CLASS_COLORS_LIST)
plot_constraints(CONSTRAINTS, overlapping=True, class_colors_list=CLASS_COLORS_LIST)
plot_pca_loadings(IRIS_FEATURES, IRIS.feature_names)

## Single Counterfactual Example with Custom Rules

In [ ]:
# Define custom rules for this example
custom_rules = {
    'petal width (cm)': 'non_decreasing',
    'petal length (cm)': 'no_change',
    'sepal length (cm)': 'non_increasing',
    'sepal width (cm)': 'non_decreasing'
}

print(f"Generating counterfactual with rules: {custom_rules}\n")
print(f"Constraints: {CONSTRAINTS}\n")
# Create a new CounterFactualModel instance with custom rules
cf_dpg_custom = CounterFactualModel(MODEL, CONSTRAINTS)
cf_dpg_custom.dict_non_actionable = custom_rules

# Generate the counterfactual with retries
custom_counterfactual = None
for attempt in range(15):
    print(f"Attempt {attempt + 1}/15")
    custom_counterfactual = cf_dpg_custom.generate_counterfactual(
        ORIGINAL_SAMPLE, 
        TARGET_CLASS, 
        INITIAL_POPULATION_SIZE, 
        MAX_GENERATIONS
    )
    if custom_counterfactual is not None:
        print(f"✓ Counterfactual successfully generated on attempt {attempt + 1}!")
        break

if custom_counterfactual is not None:
    # Create explainer for metrics
    custom_explainer = CounterFactualExplainer(cf_dpg_custom, ORIGINAL_SAMPLE, custom_counterfactual, TARGET_CLASS)
    
    # Display explanations
    print("\n" + "="*80)
    print("FEATURE MODIFICATIONS")
    print("="*80)
    feature_mods = custom_explainer.explain_feature_modifications()
    for mod in feature_mods:
        print(f"Feature '{mod['feature_name']}': {mod['old_value']} → {mod['new_value']}")
    
    print("\n" + "="*80)
    print("CONSTRAINTS RESPECT")
    print("="*80)
    print(custom_explainer.check_constraints_respect())
    
    print("\n" + "="*80)
    print("STOPPING CRITERIA")
    print("="*80)
    print(custom_explainer.explain_stopping_criteria())
    
    print("\n" + "="*80)
    print("FINAL RESULTS")
    print("="*80)
    print(custom_explainer.summarize_final_results())
    
    # Generate and display visualizations
    print("\n" + "="*80)
    print("VISUALIZATIONS")
    print("="*80)
    
    # Heatmap comparison
    custom_viz_heatmap = plot_sample_and_counterfactual_heatmap(
        ORIGINAL_SAMPLE, 
        ORIGINAL_SAMPLE_PREDICTED_CLASS, 
        custom_counterfactual, 
        MODEL.predict(pd.DataFrame([custom_counterfactual])), 
        custom_rules
    )
    display(custom_viz_heatmap)
    
    # Feature comparison
    custom_viz_comparison = plot_sample_and_counterfactual_comparison(
        MODEL, 
        ORIGINAL_SAMPLE, 
        SAMPLE_DATAFRAME, 
        custom_counterfactual, 
        CONSTRAINTS,
        CLASS_COLORS_LIST
    )
    display(custom_viz_comparison)
    
    # Fitness evolution
    custom_viz_fitness = cf_dpg_custom.plot_fitness()
    display(custom_viz_fitness)
    
    # PCA with single counterfactual
    cf_features_df_single = pd.DataFrame([custom_counterfactual])
    custom_viz_pca = plot_pca_with_counterfactuals(
        MODEL, 
        pd.DataFrame(IRIS_FEATURES), 
        IRIS_LABELS, 
        ORIGINAL_SAMPLE, 
        cf_features_df_single
    )
    display(custom_viz_pca)
    
    # Pairwise plot
    custom_viz_pairwise = plot_pairwise_with_counterfactual_df(
        MODEL, 
        IRIS_FEATURES, 
        IRIS_LABELS, 
        ORIGINAL_SAMPLE, 
        cf_features_df_single
    )
    display(custom_viz_pairwise)
    
else:
    print("✗ Failed to generate counterfactual after 15 attempts.")

## Generate Counterfactuals with All Rule Combinations

In [ ]:
from ipywidgets import IntProgress, HTML, HBox, Output, Layout
from IPython.display import display, clear_output
import sys

# Create progress widget with fixed positioning
progress_widget = IntProgress(
    value=0,
    min=0,
    max=NUMBER_OF_COMBINATIONS_TO_TEST,
    description='',
    bar_style='info',
    orientation='horizontal',
    layout=Layout(width='500px')
)

progress_text = HTML(value='<b>Progress: 0 / ' + str(NUMBER_OF_COMBINATIONS_TO_TEST) + '</b>')
progress_container = HBox([progress_widget, progress_text], layout=Layout(width='100%', padding='10px'))

# Output area for scrolling logs
logs_output = Output(layout=Layout(height='400px', overflow_y='auto', border='1px solid black', padding='10px'))

# Display the fixed progress widget and scrolling output area
display(progress_container)
display(logs_output)

counterfactuals_df_combinations = []
visualizations = []

for combination_num, combination in enumerate(RULES_COMBINATIONS[:NUMBER_OF_COMBINATIONS_TO_TEST]):
    # Update progress widget
    progress_widget.value = combination_num + 1
    progress_text.value = f'<b>Progress: {combination_num + 1} / {NUMBER_OF_COMBINATIONS_TO_TEST}</b>'
    
    dict_non_actionable = dict(zip(FEATURES_NAMES, combination))
    counterfactuals_df_replications = []
    combination_viz = {
        'label': combination,
        'pairwise': None,
        'pca': None,
        'replication': []
    }
    
    # Track if we should skip this combination
    skip_combination = False
    
    for replication in range(NUMBER_OF_REPLICATIONS_PER_COMBINATION):
        # If 3rd replication failed, skip the rest of this combination
        if skip_combination:
            break
            
        with logs_output:
            print(f"\nCombination {combination_num + 1}/{NUMBER_OF_COMBINATIONS_TO_TEST}: {dict_non_actionable}, Replication: {replication + 1}/{NUMBER_OF_REPLICATIONS_PER_COMBINATION}")
        
        COUNTERFACTUAL_DPG = CounterFactualModel(MODEL, CONSTRAINTS)
        COUNTERFACTUAL_DPG.dict_non_actionable = dict_non_actionable

        counterfactual = COUNTERFACTUAL_DPG.generate_counterfactual(ORIGINAL_SAMPLE, TARGET_CLASS, INITIAL_POPULATION_SIZE, MAX_GENERATIONS)
        if (counterfactual == None):
            # If 3rd replication (index 2) fails, skip the rest of the combination
            if replication == 2:
                with logs_output:
                    print(f"Skipping remaining replications for combination {combination_num + 1} due to failed 3rd replication")
                skip_combination = True
            continue
        
        with logs_output:
            print(f"Counterfactual found for combination: {dict_non_actionable}")

        # Store counterfactual and model in replication_viz object
        replication_viz = {
            'counterfactual': counterfactual,
            'cf_model': COUNTERFACTUAL_DPG,  # Store the model so we can access fitness data later
            'visualizations': [],
            'explanations': {}  # Changed to dictionary
        }
        combination_viz['replication'].append(replication_viz)

        # Prepare data for DataFrame
        cf_data = counterfactual.copy()
        cf_data.update({'Rule_' + k: v for k, v in dict_non_actionable.items()})
        cf_data['Replication'] = replication + 1
        counterfactuals_df_replications.append(cf_data)
    
    # Convert replications to DataFrame (no plotting yet)
    if counterfactuals_df_replications:
        counterfactuals_df_replications = pd.DataFrame(counterfactuals_df_replications)
        
        # Add all replications to the overall combinations list
        counterfactuals_df_combinations.extend(counterfactuals_df_replications.to_dict('records'))
    
    if(combination_viz['replication']):
        visualizations.append(combination_viz)


# Convert all combinations to DataFrame
counterfactuals_df_combinations = pd.DataFrame(counterfactuals_df_combinations)

with logs_output:
    print("\n✓ Counterfactual generation complete!")


## Generate Visualizations for Counterfactuals

In [ ]:
# Create progress widget for visualization generation
viz_progress_widget = IntProgress(
    value=0,
    min=0,
    max=len(visualizations),
    description='Visualizations:',
    bar_style='info',
    orientation='horizontal',
    layout=Layout(width='500px')
)

viz_progress_text = HTML(value='<b>Progress: 0 / ' + str(len(visualizations)) + '</b>')
viz_progress_container = HBox([viz_progress_widget, viz_progress_text], layout=Layout(width='100%', padding='10px'))

# Output area for scrolling logs
viz_logs_output = Output(layout=Layout(height='400px', overflow_y='auto', border='1px solid black', padding='10px'))

# Display the fixed progress widget and scrolling output area
display(viz_progress_container)
display(viz_logs_output)

# Iterate over all combinations and generate visualizations
for combination_idx, combination_viz in enumerate(visualizations):
    # Update progress widget
    viz_progress_widget.value = combination_idx + 1
    viz_progress_text.value = f'<b>Progress: {combination_idx + 1} / {len(visualizations)}</b>'
    
    dict_non_actionable = dict(zip(FEATURES_NAMES, combination_viz['label']))
    
    with viz_logs_output:
        print(f"\nGenerating visualizations for combination {combination_idx + 1}/{len(visualizations)}: {dict_non_actionable}")
    
    # Generate visualizations for each replication
    for replication_idx, replication_viz in enumerate(combination_viz['replication']):
        counterfactual = replication_viz['counterfactual']
        COUNTERFACTUAL_DPG = replication_viz['cf_model']  # Use the stored model instead of regenerating
        \
        with viz_logs_output:
            print(f"  Replication {replication_idx + 1}/{len(combination_viz['replication'])}")
        
        # Generate replication visualizations
        replication_visualizations = [
            plot_sample_and_counterfactual_heatmap(ORIGINAL_SAMPLE, ORIGINAL_SAMPLE_PREDICTED_CLASS, counterfactual, MODEL.predict(pd.DataFrame([counterfactual])), dict_non_actionable),
            plot_sample_and_counterfactual_comparison(MODEL, ORIGINAL_SAMPLE, SAMPLE_DATAFRAME, counterfactual,CONSTRAINTS, CLASS_COLORS_LIST),
            COUNTERFACTUAL_DPG.plot_fitness()  # Use the stored model's fitness data
        ]
        
        # Store visualizations in the replication object
        replication_viz['visualizations'] = replication_visualizations
    
    # Generate combination-level visualizations (PCA and Pairwise)
    # Extract all counterfactuals for this combination
    counterfactuals_list = [rep['counterfactual'] for rep in combination_viz['replication']]
    cf_features_df = pd.DataFrame(counterfactuals_list)
    
    combination_viz['pairwise'] = plot_pairwise_with_counterfactual_df(MODEL, IRIS_FEATURES, IRIS_LABELS, ORIGINAL_SAMPLE, cf_features_df)
    combination_viz['pca'] = plot_pca_with_counterfactuals(MODEL, pd.DataFrame(IRIS_FEATURES), IRIS_LABELS, ORIGINAL_SAMPLE, cf_features_df)

with viz_logs_output:
    print("\n✓ Visualization generation complete!")


## Metrics

In [ ]:
# Create progress widget for metrics generation
metrics_progress_widget = IntProgress(
    value=0,
    min=0,
    max=len(visualizations),
    description='Metrics:',
    bar_style='info',
    orientation='horizontal',
    layout=Layout(width='500px')
)

metrics_progress_text = HTML(value='<b>Progress: 0 / ' + str(len(visualizations)) + '</b>')
metrics_progress_container = HBox([metrics_progress_widget, metrics_progress_text], layout=Layout(width='100%', padding='10px'))

# Output area for scrolling logs
metrics_logs_output = Output(layout=Layout(height='400px', overflow_y='auto', border='1px solid black', padding='10px'))

# Display the fixed progress widget and scrolling output area
display(metrics_progress_container)
display(metrics_logs_output)

# Iterate over all combinations and generate metrics/explainers
for combination_idx, combination_viz in enumerate(visualizations):
    # Update progress widget
    metrics_progress_widget.value = combination_idx + 1
    metrics_progress_text.value = f'<b>Progress: {combination_idx + 1} / {len(visualizations)}</b>'
    
    dict_non_actionable = dict(zip(FEATURES_NAMES, combination_viz['label']))
    
    with metrics_logs_output:
        print(f"\nGenerating metrics for combination {combination_idx + 1}/{len(visualizations)}: {dict_non_actionable}")
    
    # Generate metrics for each replication
    for replication_idx, replication_viz in enumerate(combination_viz['replication']):
        counterfactual = replication_viz['counterfactual']
        COUNTERFACTUAL_DPG = replication_viz['cf_model']
        
        with metrics_logs_output:
            print(f"  Replication {replication_idx + 1}/{len(combination_viz['replication'])}")
        
        # Generate individual explainer metrics
        EXPLAINER = CounterFactualExplainer(COUNTERFACTUAL_DPG, ORIGINAL_SAMPLE, counterfactual, TARGET_CLASS)
        
        # Store explanations as key-value pairs (dictionary)
        replication_viz['explanations'] = {
            'Feature Modifications': EXPLAINER.explain_feature_modifications(),
            'Constraints Respect': EXPLAINER.check_constraints_respect(),
            'Stopping Criteria': EXPLAINER.explain_stopping_criteria(),
            'Final Results': EXPLAINER.summarize_final_results()
        }

with metrics_logs_output:
    print("\n✓ Metrics generation complete!")

In [ ]:
# Display visualizations with dropdown picker for combination selection
from ipywidgets import Dropdown, Output, VBox, HTML, Layout, IntSlider, HBox

# Define color mapping for rules
RULE_COLORS = {
    'no_change': '#FF6B6B',  # Red
    'non_increasing': '#4ECDC4',  # Teal
    'non_decreasing': '#123456'  # blue
}

# Create combination slider
num_combinations = len(visualizations)
combination_slider = IntSlider(
    value=0,
    min=0,
    max=max(0, num_combinations - 1),
    step=1,
    description='Combination:',
    layout=Layout(width='500px')
)

# Create combined label to show features and current rules
def create_combined_label(combination_idx):
    """Create a label combining features and their corresponding rules with color coding"""
    if not visualizations:
        return "<b>No visualizations available</b>"
    
    rules_tuple = visualizations[combination_idx]['label']
    label_parts = []
    for feat, rule in zip(FEATURES_NAMES, rules_tuple):
        color = RULE_COLORS.get(rule, '#000000')
        label_parts.append(f"<b>{feat}=</b><span style='color: {color}; font-weight: bold;'>{rule}</span>")
    
    return "<br>".join(label_parts)

combined_label = HTML(value=create_combined_label(0))

# Create replication slider
replication_slider = IntSlider(
    value=0,
    min=0,
    max=0,
    step=1,
    description='Replication:',
    layout=Layout(width='500px')
)

# Create output areas
combination_output_area = Output()  # For PCA and Pairwise plots
replication_output_area = Output()  # For replication visualizations

def display_combination_plots(combination_idx):
    """Display PCA and Pairwise plots for selected combination"""
    combination_output_area.clear_output(wait=True)
    with combination_output_area:
        combination_viz = visualizations[combination_idx]

        if combination_viz['pca'] is not None:
            display(combination_viz['pca'])

        if combination_viz['pairwise'] is not None:
            display(combination_viz['pairwise'])

def display_replication(combination_idx, replication_idx):
    """Display visualizations for selected replication"""
    replication_output_area.clear_output(wait=True)
    with replication_output_area:
        combination_viz = visualizations[combination_idx]
        replication_viz = combination_viz['replication'][replication_idx]
        
        print(f"Replication {replication_idx + 1}:")

        # Display explanations (now a dictionary)
        explanations = replication_viz.get('explanations', {})
        for explanation_name, explanation_value in explanations.items():
            print(f"\n{explanation_name}:")
            # Special formatting for Feature Modifications (list of dicts)
            if explanation_name == 'Feature Modifications' and isinstance(explanation_value, list):
                for mod in explanation_value:
                    print(f"  Feature '{mod['feature_name']}': {mod['old_value']} → {mod['new_value']}")
            else:
                print(explanation_value)

        # Use nested visualizations list
        for viz_idx, viz in enumerate(replication_viz['visualizations']):
            display(viz)

# Setup event handler for combination change
def on_combination_change(change):
    combination_idx = change['new']
    
    # Update combined label with features and rules
    combined_label.value = create_combined_label(combination_idx)
    
    # Update replication slider range
    num_replications = len(visualizations[combination_idx]['replication'])
    replication_slider.max = num_replications - 1
    replication_slider.value = 0
    
    # Update combination slider max label
    combination_slider_label.value = f"/ {combination_slider.max}"
    
    # Update replication slider max label
    replication_slider_label.value = f"/ {replication_slider.max}"
    
    # Display combination plots
    display_combination_plots(combination_idx)
    
    # Display first replication
    display_replication(combination_idx, 0)

# Setup event handler for replication change
def on_replication_change(change):
    combination_idx = combination_slider.value
    replication_idx = change['new']
    display_replication(combination_idx, replication_idx)

combination_slider.observe(on_combination_change, names='value')
replication_slider.observe(on_replication_change, names='value')

# Create slider labels for max values
combination_slider_label = HTML(value=f"/ {combination_slider.max}", layout=Layout(width='auto', margin='5px 0 0 10px'))
replication_slider_label = HTML(value=f"/ {replication_slider.max}", layout=Layout(width='auto', margin='5px 0 0 10px'))

# Wrap sliders with labels
combination_slider_with_label = HBox([combination_slider, combination_slider_label])
replication_slider_with_label = HBox([replication_slider, replication_slider_label])

# Display initial combination
on_combination_change({'new': 0})

# Show combined label, sliders and output areas
display(VBox([combined_label, combination_slider_with_label, combination_output_area, replication_slider_with_label, replication_output_area]))


## Summary Table of All Counterfactuals

In [ ]:
import pandas as pd
import json
from IPython.display import display
from ipywidgets import IntSlider, VBox, HBox, Output, Button, Layout, HTML as HTMLWidget

# Build a comprehensive table from the visualizations data structure
table_data = []

for combination_idx, combination_viz in enumerate(visualizations):
    # Get the rules for this combination
    rules_tuple = combination_viz['label']
    rules_dict = dict(zip(FEATURES_NAMES, rules_tuple))
    
    # Iterate through each replication
    for replication_idx, replication_viz in enumerate(combination_viz['replication']):
        row = {
            'Combination': combination_idx + 1,
            'Replication': replication_idx + 1,
        }
        
        # Add rule columns
        for feature_name in FEATURES_NAMES:
            row[f'Rule_{feature_name}'] = rules_dict[feature_name]
        
        # Add counterfactual feature values
        counterfactual = replication_viz['counterfactual']
        for feature_name in FEATURES_NAMES:
            row[f'CF_{feature_name}'] = counterfactual.get(feature_name, None)
        
        # Add counts of visualizations and explanations
        row['Num_Visualizations'] = len(replication_viz.get('visualizations', []))
        row['Num_Explanations'] = len(replication_viz.get('explanations', {}))
        
        # Add explanation content from dictionary
        explanations = replication_viz.get('explanations', {})
        for explanation_name, explanation_value in explanations.items():
            # Special handling for Feature Modifications (list of dicts)
            if explanation_name == 'Feature Modifications' and isinstance(explanation_value, list):
                # Format as readable text: "feature, old => new (delta)"
                formatted_mods = []
                for mod in explanation_value:
                    feature_name = str(mod['feature_name'])
                    old_val = float(mod['old_value'])
                    new_val = float(mod['new_value'])
                    delta = new_val - old_val
                    formatted_mods.append(f"{feature_name}, {old_val} => {new_val} ({delta:+.2f})")
                # Join with newlines for display
                row[explanation_name] = "\n".join(formatted_mods)
            else:
                row[explanation_name] = explanation_value
        
        # Extract any other keys that might exist in replication_viz
        for key in replication_viz.keys():
            if key not in ['counterfactual', 'cf_model', 'visualizations', 'explanations']:
                row[key] = replication_viz[key]
        
        table_data.append(row)

# Create DataFrame
summary_df = pd.DataFrame(table_data)

# Display summary statistics
print(f"Total Combinations: {len(visualizations)}")
print(f"Total Replications: {len(summary_df)}")
if len(visualizations) > 0:
    print(f"Average Replications per Combination: {len(summary_df) / len(visualizations):.2f}")
print("\n")

# Configure pandas display options for better table viewing
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)  # Set a fixed width to enable proper scrolling
pd.set_option('display.max_colwidth', 400)

# Create interactive navigation
rows_per_page = 10
total_rows = len(summary_df)
total_pages = (total_rows + rows_per_page - 1) // rows_per_page

# Page slider
page_slider = IntSlider(
    value=1,
    min=1,
    max=max(1, total_pages),
    step=1,
    description='Page:',
    layout=Layout(width='400px')
)

page_label = HTMLWidget(f'<b>of {total_pages}</b>', layout=Layout(width='auto', margin='5px 0 0 10px'))

# Navigation buttons
prev_button = Button(description='← Previous', layout=Layout(width='100px'))
next_button = Button(description='Next →', layout=Layout(width='100px'))

# Output area for table
table_output = Output(layout=Layout(width='100%', overflow_x='auto'))

def display_page(page_num):
    """Display a specific page of the table"""
    table_output.clear_output(wait=True)
    with table_output:
        start_idx = (page_num - 1) * rows_per_page
        end_idx = min(start_idx + rows_per_page, total_rows)
        
        page_df = summary_df.iloc[start_idx:end_idx]
        
        # Display page info and table
        print(f"\nShowing rows {start_idx + 1} to {end_idx} of {total_rows}\n")
        print("=" * 150)
        display(page_df)
        print("=" * 150)

def on_page_change(change):
    """Handle page slider changes"""
    display_page(change['new'])

def on_prev_click(b):
    """Handle previous button click"""
    if page_slider.value > 1:
        page_slider.value -= 1

def on_next_click(b):
    """Handle next button click"""
    if page_slider.value < total_pages:
        page_slider.value += 1

# Attach event handlers
page_slider.observe(on_page_change, names='value')
prev_button.on_click(on_prev_click)
next_button.on_click(on_next_click)

# Display initial page
display_page(1)

# Layout controls
navigation_controls = HBox([
    prev_button, 
    page_slider, 
    page_label,
    next_button
], layout=Layout(justify_content='center', padding='10px'))

# Display title and everything
print("\n" + "=" * 150)
print("DETAILED COUNTERFACTUAL SUMMARY TABLE")
print("=" * 150)
display(VBox([navigation_controls, table_output]))